In [3]:
import pandas as pd
import numpy as np
import requests
import plotly
from plotly.graph_objs import *
import networkx as nx
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
# pd.options.display.max_columns = None

### Import bibliometrics data in python

In [40]:
c1 = pd.read_excel('todos.xlsx', sheet_name='Account')
columns = pd.read_csv('identificadores.csv')

In [153]:
#Pass dataframe and dict
cd = cleaningDatabase(c1, columns)

#replace strings for lowercase and remove spaces        
cd.lowerSpaceValues(['Author Keywords', 'Document Title'])

#Get all values in columns that are empty or null
# cd.getNullValues()

In [115]:
print('Análise Descritiva')
for x in cd.df[['Publication Name','Language' ,'Document Type']].columns:
    print('-------------------')
    print(cd.df[x].value_counts().head())

Análise Descritiva
-------------------
JOURNAL OF CLEANER PRODUCTION                     114
INTERNATIONAL JOURNAL OF LIFE CYCLE ASSESSMENT     53
PLOS ONE                                           35
SCIENCE OF THE TOTAL ENVIRONMENT                   34
ECOLOGICAL MODELLING                               34
Name: Publication Name, dtype: int64
-------------------
English    1984
Russian       2
Name: Language, dtype: int64
-------------------
Article                       1895
Article; Proceedings Paper      88
Article; Book Chapter            3
Name: Document Type, dtype: int64


In [142]:
#Create lista wiht author in each line
lev = ListEachValue(cd.df, 'Document Title', 'Authors', ';')

In [146]:
#Show daframe created
lev.created.head()

,Authors,title
0,"Gao, GY",determining spatio-temporal variations of ecol...
1,"Shen, Q",determining spatio-temporal variations of ecol...
2,"Zhang, Y",determining spatio-temporal variations of ecol...
3,"Pan, NQ",determining spatio-temporal variations of ecol...
4,"Ma, Y",determining spatio-temporal variations of ecol...


In [148]:
grouped = lev.created.groupby(lev.created.columns[0]).count().reset_index()
grouped.sort_values('title', ascending=False).head()

,Authors,title
2847,"Laurie, CC",8
3305,"Margni, M",8
3417,"McKinlay, JB",7
1835,"Gogarten, SM",7
4623,"Sala, S",7


In [151]:
class cleaningDatabase:
    #Init executing function and replace columsn
    def __init__(self, df, col):
        self.col = col
        self.df = df.rename(columns=renameColuns)

    #function for replace key by value
    def renameColuns(self,x):
        col = self.col
        #if the change has already been made 
        if(x in col['Value'].values):
            return x.replace('\r\n', '')
        if(x in col['Key'].values):
            return col[col['Key'] == x]['Value'].iloc[0].replace('\r\n', '')
        else:
            return x
    
    #replace strings for lowercase and remove spaces        
    def lowerSpaceValues(self, column):
        if(type(column) == list):
            for y in column:
                self.df[y] = self.df[y].apply(lambda x: x.strip().lower() if type(x) != float else x)
        else:
            self.df[column] = self.df[column].apply(lambda x: x.strip().lower() if type(x) != float else x)
    
    #Get all values in columns that are empty or null
    def getNullValues(self):
        df = self.df
        valoresNulos = [(((len(df) - df.count())  / len(df)) * 100), len(df) - df.count()]
        dfnulos = pd.DataFrame([valoresNulos[0].apply(lambda x: ("{0:.2f} %".format(x))).values,valoresNulos[1].values], columns=df.columns, index=['Relativo','Absoluto']).transpose()
        return(dfnulos)

In [152]:
class ListEachValue:
    # {variableX} (unique) and {variableY} (with separetor) in rows 
    def __init__(self,df,variableX:str, variableY:str, sep:str):
        self.df = df
        self.variableX = variableX
        self.variableY = variableY
        self.sep = sep
        self.created = self.createNewDf(df)
    
    #Create new dataframe for list all cods of {variableX} (unique) and {variableY} (with separetor) in rows 
    def createNewDf(self,df):
        i = 0
        key = 0
        edges = pd.DataFrame()
        for index, row in df.iterrows():    
            if(self.sep in row[self.variableY]):    
                for x in row[self.variableY].split(self.sep):
                    edges = edges.append(pd.DataFrame({self.variableY: x, 'title': 1, 'title': row[self.variableX]}, index=[0]), ignore_index=True, sort=False)            
            else:
                edges= edges.append(pd.DataFrame({self.variableY: row[self.variableY], 'title': 1, 'title': row[self.variableX]}, index=[0]), ignore_index=True, sort=False)                
            i = i + 1
        return edges